In [55]:
import pandas as pd
import glob
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")
os.environ["SCIPY_ARRAY_API"] = "1"

In [56]:
# путь к папке
folder = "data/wells"

# ищем все csv-файлы
all_csv = glob.glob(os.path.join(folder, "*.csv"))

# читаем и объединяем
df_0 = pd.concat((pd.read_csv(f) for f in all_csv), ignore_index=True)
print(f"Считано файлов: {len(all_csv)}")
# переименуем для удобства
df_0 = df_0.rename(columns={"BBLS_OIL_COND":"oil", "MCF_GAS": 'gas', "BBLS_WTR":"water", "API_WellNo":"well_name", "RptDate":"date", "DAYS_PROD":"days_prod"})
# преобразуем колонку RptDate к datetime
df_0["date"] = pd.to_datetime(df_0["date"])

Считано файлов: 16492


In [57]:
df = df_0.drop(columns=["Lease_Unit", "Formation"])
print(df.shape)

(4679715, 6)


In [58]:
# важно, чтобы внутри каждой скважины ряды шли строго по времени
df = df.sort_values(by=["well_name", "date"]).reset_index(drop=True)

In [59]:
# Есть записи с отрицательными дебитами. Исключим их, так как работа будет вестись только с добывающими скважинами
df = df[(df['oil'] >= 0) & (df['gas'] >= 0) & (df['water'] >= 0)]

In [60]:
# -*- coding: utf-8 -*-
"""
Production Behavior Manifold (PBM) — Предобработка профилей (Шаг 1)
------------------------------------------------------------------
Этот блок определяет конфиг и функции предобработки для датафрейма df
со столбцами:
['well_name', 'date', 'oil', 'water', 'gas', 'days_prod']

Основные шаги:
- Приведение к среднесуточным дебитам (учёт days_prod, иначе — дни в месяце)
- Единая месячная сетка и агрегация
- Фильтры качества и робастная обработка выбросов
- Сглаживание (Savitzky–Golay)
- Производные каналы (wc, gor), нормализация по пику
- Выравнивание времени (t=0 — старт производства)
- Усечение/паддинг до горизонта T
- Подготовка длинного и «tensor» представления

Код написан так, чтобы безопасно выполняться даже без df в окружении.
"""

from tools.preprocessing import *

if 'df' in globals():
    print("Обнаружен df в окружении. Запускаю предобработку с дефолтным конфигом...")
    cfg = PreprocConfig()
    out = preprocess_profiles(df, cfg)
    # Покажем заголовок long-таблицы как предварительный просмотр
    head_preview = out["panel_long"].head(12).copy()
    try:
        # Если доступна вспомогательная функция отображения таблиц — используем её
        import caas_jupyter_tools
        caas_jupyter_tools.display_dataframe_to_user("Предпросмотр panel_long", head_preview)
    except Exception:
        # Иначе просто печатаем
        print(head_preview)
else:
    print("Готово. Функции предобработки загружены. Для запуска вызовите:")
    print("cfg = PreprocConfig(T=36, min_profile_months=12)")
    print("out = preprocess_profiles(df, cfg)")
    print("panel_long, X = out['panel_long'], out['X']")


Обнаружен df в окружении. Запускаю предобработку с дефолтным конфигом...
Preprocess complete.
  В итог попало скважин: 6353
  Отброшено (по причинам):


,reason,count
0,no_start_detected,8141
1,too_short(<12),180


             well_name       date  oil_sum  water_sum  gas_sum  days_sum  \
0   25-003-05007-00-00 1986-01-01    252.0     2552.0      0.0      31.0   
1   25-003-05007-00-00 1986-02-01    239.0     2295.0      0.0      28.0   
2   25-003-05007-00-00 1986-03-01    253.0     2697.0      0.0      31.0   
3   25-003-05007-00-00 1986-04-01    249.0     1883.0      0.0      30.0   
4   25-003-05007-00-00 1986-05-01    278.0     2631.0      0.0      31.0   
5   25-003-05007-00-00 1986-06-01    244.0     2086.0      0.0      30.0   
6   25-003-05007-00-00 1986-07-01    271.0     2322.0      0.0      31.0   
7   25-003-05007-00-00 1986-09-01    195.0     1625.0      0.0       0.0   
8   25-003-05007-00-00 1986-10-01    224.0     1867.0      0.0       0.0   
9   25-003-05007-00-00 1986-11-01    209.0     1742.0      0.0       0.0   
10  25-003-05007-00-00 1986-12-01    224.0     1867.0      0.0       0.0   
11  25-003-05007-00-00 1987-01-01    221.0     2066.0      0.0       0.0   

    valid_d

In [61]:
# -*- coding: utf-8 -*-
"""
Production Behavior Manifold (PBM) — Шаг 2 (признаки) и Шаг 3 (Manifold)
-----------------------------------------------------------------------
Зависимости (pip): numpy, pandas, scikit-learn, umap-learn, fastdtw, tqdm (опц.)

Ожидается, что из Шага 1 у вас уже есть объект `out` с ключами:
  - out["panel_long"]: long-таблица по скважинам (после предобработки)
  - out["X"]: np.ndarray формы [n_wells, T, C] по каналам cfg.tensor_channels
  - out["wells_used"]: список имён скважин в том же порядке, что и ось 0 в X
  - out["tensor_channels"]: список имён каналов в X (например: ["r_oil_norm","wc","gor","dr_oil_norm"])

Этот файл добавляет:
  1) Шаг 2: компактные дескрипторы профилей (side features)
  2) Шаг 3: построение низкоразмерной "карты поведения" (UMAP) двумя способами:
       a) быстрый базовый (евклид на выровненных рядах)
       b) уточнённый с FastDTW, без полной O(N^2) — через доуточнение k ближайших пар

Примечание: библиотеку dtaidistance *не* используем. Для DTW берём fastdtw.
"""
from __future__ import annotations
from tools.manifold import *
from tools.feature import *

try:
    from fastdtw import fastdtw
except Exception:
    fastdtw = None
    warnings.warn("fastdtw не найден. DTW-вариант будет недоступен, останется базовый евклид.")

# tqdm (опционально)
try:
    from tqdm import tqdm
    _TQDM = True
except Exception:
    _TQDM = False


if 'out' in globals():
    panel_long = out["panel_long"]
    X = out["X"]
    wells = out["wells_used"]
    tensor_channels = out["tensor_channels"]
    T = out["config"]["T"] if "config" in out else X.shape[1]

    # --- Шаг 2: компактные фичи
    feats = compute_side_features(panel_long, T=T)
    feats_scaled, scaler = scale_features(feats)
    print("Side features shape:", feats_scaled.shape)

    # --- Шаг 3a: быстрый UMAP по евклиду на рядах
    euclid_emb = embed_umap_euclid(
        X, tensor_channels=tensor_channels, channels=["r_oil_norm", "wc"],
        n_neighbors=30, min_dist=0.05, n_components=2, random_state=42
    )
    Z_euclid = euclid_emb.embedding
    umap_e = euclid_emb.umap_model

    # --- Шаг 3b: UMAP с FastDTW-уточнением
    cfg_m = ManifoldConfig(channels=("r_oil_norm", "wc"), fastdtw_radius=6, k_refine=40,
                           weights=(0.7, 0.3), n_neighbors=30, min_dist=0.05,
                           n_components=2, random_state=42)
    dtw_emb = embed_umap_fastdtw(
        X, tensor_channels=tensor_channels, channels=cfg_m.channels,
        cfg=cfg_m, sample_size=None  # можно None для полного набора
    )
    Z_dtw = dtw_emb.embedding
    sub_idx = np.asarray(dtw_emb.subset_indices, dtype=int)
    D = dtw_emb.distance_matrix
    info = dtw_emb.info or {}
    print("DTW-UMAP:", Z_dtw.shape, "subset size:", len(sub_idx))

    # Пример: собрать DataFrame с координатами
    df_map = pd.DataFrame({
        "well_name": np.array(wells)[sub_idx],
        "x": Z_dtw[:,0],
        "y": Z_dtw[:,1],
    })
    display(df_map.head())
else:
    print("Для примера использования требуется объект 'out' из Шага 1.")


Side features shape: (6353, 16)


Recompute DTW for 201681 pairs (radius=6): 100%|██████████| 201681/201681 [36:08<00:00, 93.01it/s] 


DTW-UMAP: (6353, 2) subset size: 6353


,well_name,x,y
0,25-003-05007-00-00,-1.406963,4.021780
1,25-003-05009-00-00,-0.983894,3.249930
2,25-003-05057-00-00,-1.031901,3.614995
3,25-003-05058-00-00,5.671078,9.652882
4,25-003-05059-00-00,8.030657,8.738324


In [62]:
from tools.clustering import *

if 'out' in globals() and 'Z_dtw' in globals() and 'sub_idx' in globals():
    panel_long = out["panel_long"]
    wells_all = out["wells_used"]
    T = out["config"].get("T", 72)
    wells_sub = np.array(wells_all)[sub_idx].tolist()

    # 1) HDBSCAN
    cfg = ClusterConfig(min_cluster_size=45, min_samples=12)
    res = cluster_hdbscan(Z_dtw, wells_sub, cfg)
    df_map = res["df_map"]
    print("Silhouette:", res["silhouette"], " DBCV:", res["dbcv"])
    print(summarize_clusters(df_map))

    # 2) Аномалии
    df_map = assign_anomaly_scores(df_map, Z_dtw, res["labels"], lof_k=35)
    # пример отбора аномалий
    anomalies = df_map.sort_values("anomaly_score", ascending=False).head(30)

    # 3) Прототипы (медианные или soft-DTW / DBA при наличии tslearn)
    protos = build_cluster_prototypes(panel_long, df_map, channels=("r_oil_s","wc","gor","r_oil_norm"), T=T, method="auto")

    # Теперь df_map можно соединить с геолого-технологическими признаками и строить объяснения.
else:
    print("Для примера нужен 'out' (Шаг 1) и 'Z_dtw, sub_idx' (Шаг 3).")


Silhouette: 0.48749613761901855  DBCV: nan
   cluster  size     share  prob_median
0       -1   885  0.139304     0.000000
1        0  1048  0.164961     1.000000
2        1  2758  0.434126     1.000000
3        2   129  0.020305     1.000000
4        3   278  0.043759     1.000000
5        4   872  0.137258     1.000000
6        5    46  0.007241     1.000000
7        6   194  0.030537     0.808419
8        7   143  0.022509     0.935686


In [63]:
%load_ext autoreload
%autoreload 2

from tools.make_reports import *

# -------------------- ПРИМЕР ИСПОЛЬЗОВАНИЯ --------------------
# (раскомментируйте и выполните после Шагов 1–4)
#
if 'out' in globals() and 'Z_dtw' in globals() and 'sub_idx' in globals() and 'res' in globals() and 'protos' in globals():
    panel_long = out["panel_long"]
    T = out["config"].get("T", 36)
    df_map = res["df_map"]
    out_dir = "reports/pbm_report_exports"  # или '/mnt/data/pbm_report_exports' в окружении чата

    # 1) Основные картинки
    map_png = save_pbm_map(Z_dtw, df_map, out_dir)
    sizes_png = save_cluster_distribution_plot(df_map, out_dir)

    # 2) Прототипы (oil_s, wc, gor, r_oil_norm)
    channels = ("r_oil_s","wc","gor","r_oil_norm")
    proto_pngs = save_cluster_prototype_plots(panel_long, df_map, protos, channels, T, out_dir)

    # 3) Таблицы CSV
    summary = summarize_clusters(df_map)
    csv_paths = export_csv_summaries(df_map, summary, out_dir, top_anoms=50)

    # 4) HTML отчёт
    report_path = build_html_report(out_dir, map_png, sizes_png, proto_pngs, df_map, summary, title="PBM Report")
    print("Готов отчёт:", report_path)
else:
    print("Для примера нужен 'out', 'Z_dtw','sub_idx','res','protos'.")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Готов отчёт: reports/pbm_report_exports/PBM_report.html


In [64]:
import importlib
import tools.make_reports  # твой .py файл без .py
importlib.reload(tools.make_reports)

<module 'tools.make_reports' from '/Users/george/Documents/GitHub/manifold_learning/tools/make_reports.py'>


# Шаг 6. Прогноз профиля по префиксу (20 → 100)

**Цель:** прогнозировать месяцы 21–100 для каждой скважины, используя только первые 20 месяцев и без утечки информации.

Подходы в этом шаге:
1. **KNN-достройка по префиксу** с амплитудным выравниванием соседей (по МНК на префиксе).
2. **Мультивыходная ElasticNet-регрессия** на компактных признаках префикса.

Метрики: RMSE и sMAPE на окне 21–100.  
Отчёты и артефакты сохраняются в `./forecast_exports`.


In [67]:
# === Шаг 6: Прогноз профиля по префиксу (20 → 100) ===
from tools.forecast import ForecastConfig, run_forecast_pipeline, save_forecast_artifacts

assert 'out' in globals(), "Требуется объект 'out' из Шага 1 (preprocess_profiles)."
panel_long = out["panel_long"].copy()
wells_used = out["wells_used"]
T = int(out["config"]["T"])
cfg_forecast = ForecastConfig(total_horizon=T, prefix_horizon=20, knn_neighbors=15)

result = run_forecast_pipeline(panel_long, wells_used, cfg_forecast)
exports = save_forecast_artifacts(result, output_dir="reports/forecast_exports", include_full_series=True)

print("Оценка моделей (suffix):")
for name, metrics in result.metrics.items():
    print(f"  {name}: RMSE={metrics['rmse']:.4f}, sMAPE={metrics['smape']:.4f}, N={metrics['n_eval']}")

print("Папка с артефактами:", exports['output_dir'])
print("Метрики:", exports.get('metrics_csv'))
print("Отчёт:", exports.get('report_html'))


KNN   → RMSE=17.0778, sMAPE=0.5110, N=5830
ENet  → RMSE=32.5558, sMAPE=1.2782, N=5830
Saved: reports/forecast_exports/metrics.csv and reports/forecast_exports/forecast_report.html


Выборка размера 2000:

KNN   → RMSE=20.3636, sMAPE=0.4884, N=1964
ENet  → RMSE=36.7395, sMAPE=1.1923, N=1964
Saved: reports/forecast_exports/metrics.csv and reports/forecast_exports/forecast_report.html